# Download Geospatial Layers

## Set Up Environment

In [1]:
import ee
from google.auth.transport.requests import AuthorizedSession
ee.Initialize()
session = AuthorizedSession(ee.data.get_persistent_credentials())

In [2]:
import geemap
from pprint import pprint

In [3]:
# Prep Map
Map = geemap.Map()
Map.setCenter(0, 70, 2)

In [4]:
geometry = ee.Geometry.Rectangle([-180, 43, 179.999999, 85], 'EPSG:4326', False)

## TerraClimate

In [5]:
# apply image IDs to each image in the collection
def setProperties(image):
    img_id = image.id();
    img_year = img_id.slice(0, 4);
    img_month = img_id.slice(4, 6);
    img_prop = image.setMulti({'ID': img_id,
                               'year': img_year,
                               'month': img_month});
    img_prop = ee.Image(img_prop);
    return img_prop;

In [6]:
TerraClimate = (
  ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .map(setProperties)
    .filterDate('1973-01-01', '2022-12-31')
  );

In [7]:
pprint(
    TerraClimate
    .first()
    .bandNames()
    .getInfo()
)

['aet',
 'def',
 'pdsi',
 'pet',
 'pr',
 'ro',
 'soil',
 'srad',
 'swe',
 'tmmn',
 'tmmx',
 'vap',
 'vpd',
 'vs']


In [8]:
years = TerraClimate.aggregate_array('year').distinct()
year_len = years.length()
year_min = years.get(0).getInfo()
year_max = years.get(year_len.subtract(1)).getInfo()
print(year_min)
print(year_max)

1973
2022


### Max

In [9]:
# Function to get annual max
def getAnnualMax(year):
    filtered = TerraClimate.filter(ee.Filter.eq('year', year))
    return filtered.select(['tmmx', 'pr', 'ro', 'swe']).max().set('year', year)

In [10]:
TerraClimateAnnualMax = ee.ImageCollection(years.map(getAnnualMax))
TerraClimateMax = TerraClimateAnnualMax.reduce(ee.Reducer.mean()).clip(geometry)

In [11]:
tempMaxParams = {
    'min': 5, 
    'max': 350, 
    'palette': ['FFFFCC', 'FF0000']
};
precipMaxParams = {
    'min': 0, 
    'max': 400, 
    'palette': ['FFFFFF', '0099FF']
};
roMaxParams = {
    'min': 0, 
    'max': 500, 
    'palette': ['FFFFFF', '0000FF']
};
sweMaxParams = {
    'min': 0, 
    'max': 250, 
    'palette': ['993300', 'FFFFFF']
};

In [12]:
# Map.addLayer(TerraClimateMax.select('tmmx_mean'), tempMaxParams, 'Temp Max 1973-2022')
# Map.addLayer(TerraClimateMax.select('pr_mean'), precipMaxParams, 'Precip Max 1973-2022')
# Map.addLayer(TerraClimateMax.select('ro_mean'), roMaxParams, 'Runoff Max 1973-2022')
# Map.addLayer(TerraClimateMax.select('swe_mean'), sweMaxParams, 'SWE Max 1973-2022')

### Min

In [13]:
# Function to get annual min
def getAnnualMin(year):
    filtered = TerraClimate.filter(ee.Filter.eq('year', year))
    return filtered.select(['tmmn']).min().set('year', year)

In [14]:
TerraClimateAnnualMin = ee.ImageCollection(years.map(getAnnualMin))
TerraClimateMin = TerraClimateAnnualMin.reduce(ee.Reducer.mean()).clip(geometry)

In [15]:
tempMinParams = {
    'min': -400, 
    'max': 0, 
    'palette': ['00CCFF', 'FFFFFF']
};

In [16]:
# Map.addLayer(TerraClimateMin.select('tmmn_mean'), tempMinParams, 'Temp Min 1973-2022')

### Sum

In [17]:
# Function to get annual sum
def getAnnualSum(year):
    filtered = (
        TerraClimate
        .filter(ee.Filter.eq('year', year))
        .filter(ee.Filter.And(
            ee.Filter.greaterThanOrEquals('month', '05'), 
            ee.Filter.lessThanOrEquals('month', '09'))
               )
    )
    return filtered.select(['pr', 'ro', 'srad']).sum().set('year', year)

In [18]:
TerraClimateAnnualSum = ee.ImageCollection(years.map(getAnnualSum))
TerraClimateSum = TerraClimateAnnualSum.reduce(ee.Reducer.mean()).clip(geometry)

In [19]:
precipSumParams = {
    'min': 0, 
    'max': 1000, 
    'palette': ['FFFFFF', '0099FF']
};
roSumParams = {
    'min': 0, 
    'max': 1000, 
    'palette': ['FFFFFF', '0000FF']
};
sradSumParams = {
    'min': 6000, 
    'max': 11000, 
    'palette': ['000000', 'FFEE66']
};

In [20]:
# Map.addLayer(TerraClimateSum.select('pr_mean'), precipSumParams, 'Precip Sum 1973-2022')
# Map.addLayer(TerraClimateSum.select('ro_mean'), roSumParams, 'Runoff Sum 1973-2022')
# Map.addLayer(TerraClimateSum.select('srad_mean'), sradSumParams, 'Solar Rad Sum 1973-2022')

In [21]:
# # Unexpected line in solar radiation at 180th meridian is an issue in the underlying data
# # not a problem with processing
# Map.addLayer(
#     TerraClimate.filter(ee.Filter.eq('year', '2022')).filter(ee.Filter.eq('month', '06')).select('srad'), 
#     {
#         'min': 1000, 
#         'max': 3000, 
#         'palette': ['000000', 'FFEE66']
#     },
#     'Solar Rad June 2022'
# )

### Mean

In [22]:
# Function to get annual mean
def getAnnualMean(year):
    filtered = TerraClimate.filter(ee.Filter.eq('year', year))
    return filtered.select(['soil']).mean()

In [23]:
TerraClimateAnnualMean = ee.ImageCollection(years.map(getAnnualMean))
TerraClimateMean = TerraClimateAnnualMean.reduce(ee.Reducer.mean()).clip(geometry)

In [24]:
soilMeanParams = {
    'min': 0, 
    'max': 1000, 
    'palette': ['993300', '00CCFF']
};

In [25]:
# Map.addLayer(TerraClimateMean.select('soil_mean'), soilMeanParams, 'Soil Moisture Mean 1973-2022')

### Trends

In [26]:
def addVariables(image):
    time = ee.Number.parse(image.get('year')).subtract(ee.Number.parse(TerraClimateAnnualMax.first().get('year')))
    return (
        image
        .addBands(ee.Image(time).rename('t').float())
        .addBands(ee.Image.constant(1))
        )
    
TerraClimateAnnualMax = (
    TerraClimateAnnualMax
    .map(addVariables)
)

TerraClimateAnnualMin = (
    TerraClimateAnnualMin
    .map(addVariables)
)

TerraClimateAnnualSum = (
    TerraClimateAnnualSum
    .map(addVariables)
)

independents = ee.List(['constant', 't'])

#### Tmax

In [27]:
dependent = ee.String('tmmx')
tmaxTrend = (
    TerraClimateAnnualMax.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
tmaxCoefficients = (
    tmaxTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['tmax_b0', 'tmax_b1']])
    .clip(geometry)
)

In [28]:
# This will show all of the outputs from the model (coefficients b0, b1 and residuals)
# Map.addLayer(tmaxTrend, {}, 'Tmax Model Output')

# Map with trend (b1) only
# deg C/decade (original tmmx is in deg C * 10)
# Map.addLayer(
#     tmaxCoefficients.select('tmax_b1'), 
#     {'min': -2, 'max': 2, 'palette': ['000066', '0000FF', 'FFFFFF', 'FF0000', '990000']}, 
#     'Tmax Trend'
# )

#### Tmin

In [29]:
dependent = ee.String('tmmn')
tminTrend = (
    TerraClimateAnnualMin.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
tminCoefficients = (
    tminTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['tmin_b0', 'tmin_b1']])
    .clip(geometry)
)

In [30]:
# Map with trend (b1) only
# deg C/decade (original tmmx is in deg C * 10)
# Map.addLayer(
#     tminCoefficients.select('tmin_b1'), 
#     {'min': -3, 'max': 3, 'palette': ['000066', '0000FF', 'FFFFFF', 'FF0000', '990000']}, 
#     'Tmin Trend'
# )

#### Precipitation

In [31]:
dependent = ee.String('pr')
precipTrend = (
    TerraClimateAnnualSum.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
precipCoefficients = (
    precipTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['precip_b0', 'precip_b1']])
    .clip(geometry)
)

In [32]:
# Map with trend (b1) only
# mm/yr
# Map.addLayer(
#     precipCoefficients.select('precip_b1'), 
#     {'min': -4, 'max': 4, 'palette': ['990000', 'FF0000', 'FFFFFF', '0000FF', '000066']}, 
#     'Precip Trend'
# )

#### Run-Off

In [33]:
dependent = ee.String('ro')
runoffTrend = (
    TerraClimateAnnualSum.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
runoffCoefficients = (
    runoffTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['runoff_b0', 'runoff_b1']])
    .clip(geometry)
)

In [34]:
# Map with trend (b1) only
# mm/yr
# Map.addLayer(
#     runoffCoefficients.select('runoff_b1'), 
#     {'min': -8, 'max': 8, 'palette': ['990000', 'FF0000', 'FFFFFF', '0000FF', '000066']}, 
#     'Run-Off Trend'
# )

#### Snow Water Equivalent

In [35]:
dependent = ee.String('swe')
sweTrend = (
    TerraClimateAnnualMax.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
sweCoefficients = (
    sweTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['swe_b0', 'swe_b1']])
    .clip(geometry)
)

In [36]:
# Map with trend (b1) only
# mm/yr
# Map.addLayer(
#     sweCoefficients.select('swe_b1'), 
#     {'min': -6, 'max': 6, 'palette': ['990000', 'FF0000', 'FFFFFF', '0000FF', '000066']}, 
#     'SWE Trend'
# )

## Seasonality

In [37]:
tempSeasonality = (
    TerraClimateMax
    .select('tmmx_mean')
    .subtract(TerraClimateMin.select('tmmn_mean'))
    .clip(geometry)
)

In [38]:
tempSeasonalityParams = {
    'min': 100, 
    'max': 800, 
    'palette': ['FFFFFF', '000000']
};

In [39]:
# Map.addLayer(tempSeasonality, tempSeasonalityParams, 'Temp Seasonality 1973-2022')

## SoilGrids

In [40]:
# select first 5 bands which correspond to top meter of soil
clay = (
    ee.Image("projects/soilgrids-isric/clay_mean")
    .select(1, 2, 3, 4, 5)
    .reduce(ee.Reducer.mean())
    .clip(geometry)
);
sand = (
    ee.Image("projects/soilgrids-isric/sand_mean")
    .select(1, 2, 3, 4, 5)
    .reduce(ee.Reducer.mean())
    .clip(geometry)
);
silt = (
    ee.Image("projects/soilgrids-isric/silt_mean")
    .select(1, 2, 3, 4, 5)
    .reduce(ee.Reducer.mean())
    .clip(geometry)
);
ocs = (
    ee.Image("projects/soilgrids-isric/ocs_mean") # organic carbon stock
    .clip(geometry)
);

In [41]:
# Map.addLayer(clay, {'min': 50, 'max': 300 , 'palette': ['000000', 'FFFFFF']}, 'Clay')

In [42]:
# Map.addLayer(sand, {'min': 50, 'max': 500 , 'palette': ['000000', 'FFFFFF']}, 'Sand')

In [43]:
# Map.addLayer(silt, {'min': 50, 'max': 500 , 'palette': ['000000', 'FFFFFF']}, 'Silt')

In [44]:
# Map.addLayer(ocs, {'min': 0, 'max': 100 , 'palette': ['000000', 'FFFFFF']}, 'Organic Carbon Stock')

## MODIS LST

In [45]:
# apply image IDs to each image in the collection
def setProperties(image):
    img_id = image.id();
    img_year = img_id.slice(0, 4);
    img_month = img_id.slice(5, 7);
    img_prop = image.setMulti({'ID': img_id,
                               'year': img_year,
                               'month': img_month});
    img_prop = ee.Image(img_prop);
    return img_prop;

In [46]:
lst = (
    ee.ImageCollection("MODIS/061/MOD21A1D")
    .filterDate('2003-01-01', '2022-12-31')
    .select('LST_1KM')
    .map(setProperties)
);

In [47]:
years = lst.aggregate_array('year').distinct()
print(years.getInfo())

['2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [48]:
# Function to get annual mean
def getGSMean(year):
    filtered = (
        lst
        .filter(ee.Filter.eq('year', year))
        .filter(ee.Filter.And(
            ee.Filter.greaterThanOrEquals('month', '05'), 
            ee.Filter.lessThanOrEquals('month', '09'))
               )
    )
    return filtered.select(['LST_1KM']).mean().set('year', year)

In [49]:
lstAnnualGSMean = ee.ImageCollection(
    years
    .map(getGSMean)
    
)
lstGSMean = (
    lstAnnualGSMean
    .reduce(ee.Reducer.mean())
    .clip(geometry)
)

In [50]:
lstParams = {
    'min': 200, 
    'max': 600, 
    'palette': ['FFFFFF', '000000']
};

In [51]:
# Map.addLayer(lstGSMean, lstParams, 'MODIS GS LST 2003-2022')

### Trend

In [52]:
def addVariables(image):
    time = ee.Number.parse(image.get('year')).subtract(ee.Number.parse(lstAnnualGSMean.first().get('year')))
    return (
        image
        .addBands(ee.Image(time).rename('t').float())
        .addBands(ee.Image.constant(1))
        )

In [53]:
lstAnnualGSMean = (
    lstAnnualGSMean
    .map(addVariables)
)

In [54]:
dependent = ee.String('LST_1KM')
lstTrend = (
    lstAnnualGSMean.select(independents.add(dependent))
    .reduce(ee.Reducer.linearRegression(independents.length(), 1))
)
lstCoefficients = (
    sweTrend.select('coefficients')
    .arrayProject([0])
    .arrayFlatten([['lst_b0', 'lst_b1']])
    .clip(geometry)
)

In [55]:
# # Map with trend (b1) only
# # mm/yr
# Map.addLayer(
#     lstCoefficients.select('lst_b1'), 
#     {'min': -6, 'max': 6, 'palette': ['990000', 'FF0000', 'FFFFFF', '0000FF', '000066']}, 
#     'LST Trend'
# )

## Topography

In [56]:
copDEM = (
    ee.ImageCollection('COPERNICUS/DEM/GLO30')
    .select('DEM')
    .mosaic()
    .setDefaultProjection('EPSG:3857',None,30)
    .clip(geometry)
);

# Calculate slope (degrees, range is [0,90)), aspect (degrees where 0=N, 90=E, etc.), hillslope[0,255].
copTerrain = (
    ee.Terrain.products(copDEM)
    .setDefaultProjection('EPSG:3857',None,30)
);
# pprint(copTerrain.bandNames().getInfo());

In [57]:
Map.addLayer(
    copTerrain.select('DEM'), 
    {'min': 0, 'max': 1000, 'palette': ['000000', 'FFFFFF']},
    'Copernicus DEM'
)

In [58]:
Map.addLayer(
    copTerrain.select('slope'), 
    {'min': 0, 'max': 45, 'palette': ['000000', 'FFFFFF']},
    'Copernicus Slope'
)

In [59]:
Map.addLayer(
    copTerrain.select('aspect'), 
    {'min': 0, 'max': 360, 'palette': ['000000', 'FFFFFF']},
    'Copernicus Aspect'
)

### Aggregated Variables

In [60]:
# % cover of suitable slopes
copSlope = (
    copTerrain
    .select('slope')
    .expression('b(0) < 1 ? 0 : b(0) < 20 ? 1 : 0')
    .setDefaultProjection('EPSG:4326',None,30)
    .reduceResolution(
        reducer = ee.Reducer.mean(),
        maxPixels = 35*35
    )
    .reproject(
        crs = 'EPSG:4326',
        scale = 1000
    )
    .clip(geometry)
);

pprint(copSlope.bandNames().getInfo())

['constant']


In [61]:
Map.addLayer(copSlope, {'palette': ['000000', 'FFFFFF']}, 'Slope (Categorized)')

## Map

In [62]:
Map

Map(center=[70, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Export

In [63]:
crs = 'EPSG:4326'
Map.addLayer(geometry, {'color': 'red'}, 'Export BBox')

In [64]:
# # TerraClimateMin
# task = ee.batch.Export.image.toDrive(
#     image = TerraClimateMin,
#     description = 'TerraClimateMin4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [65]:
# # TerraClimateMax
# task = ee.batch.Export.image.toDrive(
#     image = TerraClimateMax,
#     description = 'TerraClimateMax4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [66]:
# # TerraClimateSum
# task = ee.batch.Export.image.toDrive(
#     image = TerraClimateSum,
#     description = 'TerraClimateSum4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [67]:
# # TerraClimateMean
# task = ee.batch.Export.image.toDrive(
#     image = TerraClimateMean,
#     description = 'TerraClimateMean4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [68]:
# # TerraClimateTempSeasonality
# task = ee.batch.Export.image.toDrive(
#     image = tempSeasonality,
#     description = 'TerraClimateTempSeasonality4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [69]:
# # Tmax trend
# task = ee.batch.Export.image.toDrive(
#     image = tmaxCoefficients,
#     description = 'TerraClimateTmaxCoefficients4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [70]:
# # Tmin trend
# task = ee.batch.Export.image.toDrive(
#     image = tminCoefficients,
#     description = 'TerraClimateTminCoefficients4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [71]:
# # Precip trend
# task = ee.batch.Export.image.toDrive(
#     image = precipCoefficients,
#     description = 'TerraClimatePrecipCoefficients4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [72]:
# # Runoff trend
# task = ee.batch.Export.image.toDrive(
#     image = runoffCoefficients,
#     description = 'TerraClimateRunoffCoefficients4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [73]:
# # SWE trend
# task = ee.batch.Export.image.toDrive(
#     image = sweCoefficients,
#     description = 'TerraClimateSWECoefficients4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [74]:
# # Clay
# task = ee.batch.Export.image.toDrive(
#     image = clay,
#     description = 'SoilGridsClay4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [75]:
# # Sand
# task = ee.batch.Export.image.toDrive(
#     image = sand,
#     description = 'SoilGridsSand4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [76]:
# # Silt
# task = ee.batch.Export.image.toDrive(
#     image = silt,
#     description = 'SoilGridsSilt4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [77]:
# # Organic Carbon Stock
# task = ee.batch.Export.image.toDrive(
#     image = ocs,
#     description = 'SoilGridsOCS4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [78]:
# # MODIS LST
# task = ee.batch.Export.image.toDrive(
#     image = lstGSMean,
#     description = 'MODISLSTGSMean4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [79]:
# # MODIS LST
# task = ee.batch.Export.image.toDrive(
#     image = lstCoefficients,
#     description = 'MODISLSTGSTrend4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat='GeoTIFF'
    
# )
# task.start()

In [80]:
# # Copernicus % Cover of Suitable Slopes
# task = ee.batch.Export.image.toDrive(
#     image = copSlope,
#     description = 'CopernicusSuitableSlope4326',
#     folder = 'Earth Engine Exports',
#     crs = crs,
#     region = geometry,
#     scale = 1000,
#     maxPixels = 1e13,
#     fileFormat = 'GeoTIFF',
#     shardSize = 256/2,
#     skipEmptyTiles = True
# )
# task.start()